### Train model

In [7]:
# import numpy as np
# import cv2
# import warnings
# warnings.filterwarnings('ignore')

# path = 'D:\JUPYTER\YAY\haarcascades\haarcascade_frontalface_alt.xml'
# face_cascade = cv2.CascadeClassifier(path)

# dirpath = 'D:\JUPYTER\YAY\haarcascades\stored_pics'

# name = input('Enter your name:  ')

# face_data = []
# skip = 0

# cap = cv2.VideoCapture(0)
# if not cap.isOpened():
#     print('Cannot access video')
#     exit()
    
# while True:
#     success, frame = cap.read()
    
#     if not success:
#         print('Cannot receive frame. Exiting...')
#         exit()

#     cv2.imshow('Camera', frame)
    
#     gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
#     faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    
#     if faces == ():
#         continue
#     else:
#         faces = sorted(faces, key=lambda f: f[2]^f[3])
#         for face in faces[-1:]:
#             x, y, w, h = face
#             face_section = gray[y:y+h, x:x+w]
#             face_section = cv2.resize(face_section, (100,100))
#             cv2.putText(frame, name, (x, y-30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0), 2, cv2.LINE_AA)
#             cv2.rectangle(frame, (x, y), (x+w, y+h), (0,255,0), 10)

#     cv2.imshow("Camera", frame)
    
#     if skip%10 == 0:
#         face_data.append(face_section)
#     skip += 1
        
    
#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break
   

# # 
# face_data = np.asarray(face_data)
# face_data = face_data.reshape(face_data.shape[0], -1)

# np.save(dirpath + "\\" + name + '.npy', face_data)

# cap.release()
# cv2.destroyAllWindows()

Enter your name:  papa


### Run model

In [9]:
import numpy as np
import cv2
import warnings
import os
warnings.filterwarnings('ignore')


from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=5)


cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print('Cannot access video')
    exit()

path = 'D:\JUPYTER\YAY\haarcascades\haarcascade_frontalface_alt.xml'
face_cascade = cv2.CascadeClassifier(path)

face_data = []
skip = 0
labels = []
names = {}
class_id = 0
face_section = np.zeros((100, 100), dtype='uint8')

dirpath = 'D:\JUPYTER\YAY\haarcascades\stored_pics'

for file in os.listdir(dirpath):
    if file.endswith(".npy"):
        stored_item = np.load(dirpath+'\\'+file)
        names[class_id] = file[:-4]
        face_data.append(stored_item)
        target = class_id * np.ones((stored_item.shape[0],))
        class_id += 1
        labels.append(target)

face_dataset = np.concatenate(face_data,axis = 0)
face_labels = np.concatenate(labels,axis = 0).reshape((-1,1)) 

while True:
    ret , frame = cap.read()
    gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    if ret == False:
        continue
    faces = face_cascade.detectMultiScale(gray,1.3,5)
    
    if faces == ():
        continue
    else:
#         sort acc to area
        faces = sorted(faces,key = lambda f:f[2]*f[3])
        for face in faces[-1:]:
            x,y,w,h = face

            offset = 10
            face_section = gray[y-offset:y+h+offset,x-offset:w+x+offset]
            face_section = cv2.resize(face_section,(100,100))
            
            knn.fit(face_dataset,face_labels)
            pred = knn.predict([face_section.flatten()])
            pred_name = names[int(pred)]
            cv2.putText(frame,pred_name,(x,y-30),cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0),2,cv2.LINE_AA)
            cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),10)

    cv2.imshow("camera", frame)
    
    key_pressed = cv2.waitKey(1) & 0xFF
    if key_pressed == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()
